In [1]:
import pandas as pd
import numpy as np

__ 1. Processing the trip dataset __

In [2]:
df_trip = pd.read_csv('data/2011-cabi-trip-history-data-final.csv',parse_dates=[0,1])

In [3]:
df_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127471 entries, 0 to 1127470
Data columns (total 7 columns):
start_date            1127471 non-null datetime64[ns]
end_date              1127471 non-null datetime64[ns]
duration_in_sec       1127471 non-null int64
start_station_name    1127471 non-null object
end_station_name      1127471 non-null object
bike#                 1127471 non-null object
member_type           1127471 non-null object
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 60.2+ MB


In [4]:
df_trip.head()

,start_date,end_date,duration_in_sec,start_station_name,end_station_name,bike#,member_type
0,2011-03-31 23:58:00,2011-04-01 00:00:00,110,14th & Harvard St NW,16th & Harvard St NW,W00749,Registered
1,2011-03-31 23:52:00,2011-04-01 00:08:00,981,19th & L St NW,7th & Water St SW / SW Waterfront,W01048,Casual
2,2011-03-31 23:47:00,2011-03-31 23:50:00,199,Lincoln Park / 13th & East Capitol St NE,13th & H St NE,W00340,Registered
3,2011-03-31 23:45:00,2011-03-31 23:50:00,344,14th & R St NW,Massachusetts Ave & Dupont Circle NW,W00981,Registered
4,2011-03-31 23:34:00,2011-03-31 23:43:00,522,Park Rd & Holmead Pl NW,15th & P St NW,W00449,Registered


In [5]:
df_trip.index = df_trip.start_date

In [6]:
registered = {'Registered':1,'Casual':0}
casual = {'Registered':0,'Casual':1}
df_trip['cnt_reg'] =df_trip.member_type.map(registered)
df_trip['cnt_cas'] =df_trip.member_type.map(casual)

In [7]:
df_trip.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1127471 entries, 2011-03-31 23:58:00 to 2011-10-01 00:01:00
Data columns (total 9 columns):
start_date            1127471 non-null datetime64[ns]
end_date              1127471 non-null datetime64[ns]
duration_in_sec       1127471 non-null int64
start_station_name    1127471 non-null object
end_station_name      1127471 non-null object
bike#                 1127471 non-null object
member_type           1127471 non-null object
cnt_reg               1127471 non-null int64
cnt_cas               1127471 non-null int64
dtypes: datetime64[ns](2), int64(3), object(4)
memory usage: 86.0+ MB


In [8]:
df_trip.cnt_cas.sum() + df_trip.cnt_reg.sum()

1127471

In [9]:
df_trip_agg = df_trip.resample('H').agg({'start_station_name':'count',\
                                         'cnt_reg':'sum','cnt_cas':'sum','duration_in_sec':'sum'})

In [10]:
df_trip_agg.head()

,start_station_name,cnt_reg,cnt_cas,duration_in_sec
start_date,,,,
2011-01-01 00:00:00,15,12.0,3.0,7745.0
2011-01-01 01:00:00,38,31.0,7.0,21907.0
2011-01-01 02:00:00,31,26.0,5.0,14126.0
2011-01-01 03:00:00,12,9.0,3.0,6622.0
2011-01-01 04:00:00,1,1.0,0.0,605.0


In [11]:
df_trip_agg.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2011-01-01 00:00:00 to 2011-12-31 23:00:00
Freq: H
Data columns (total 4 columns):
start_station_name    8760 non-null int64
cnt_reg               8643 non-null float64
cnt_cas               8643 non-null float64
duration_in_sec       8643 non-null float64
dtypes: float64(3), int64(1)
memory usage: 662.2 KB


In [12]:
df_trip_agg = df_trip_agg.fillna(0)

In [13]:
# There're 8760 hours in 2011
df_trip_agg.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2011-01-01 00:00:00 to 2011-12-31 23:00:00
Freq: H
Data columns (total 4 columns):
start_station_name    8760 non-null int64
cnt_reg               8760 non-null float64
cnt_cas               8760 non-null float64
duration_in_sec       8760 non-null float64
dtypes: float64(3), int64(1)
memory usage: 662.2 KB


In [14]:
df_trip_agg.start_station_name.sum()

1127471

In [15]:
df_trip.duration_in_sec.sum()

825459426

In [16]:
df_trip_agg.duration_in_sec.sum()

825459426.0

In [17]:
df_trip_agg.reset_index(inplace=True)

In [18]:
df_trip_agg.rename(columns={'start_date':'date','start_station_name':'count'},inplace=True)

In [19]:
df_trip_agg['duration_in_min'] = df_trip_agg.duration_in_sec/60

In [20]:
df_trip_agg = df_trip_agg.round(0)

In [21]:
df_trip_agg.head()

,date,count,cnt_reg,cnt_cas,duration_in_sec,duration_in_min
0,2011-01-01 00:00:00,15,12.0,3.0,7745.0,129.0
1,2011-01-01 01:00:00,38,31.0,7.0,21907.0,365.0
2,2011-01-01 02:00:00,31,26.0,5.0,14126.0,235.0
3,2011-01-01 03:00:00,12,9.0,3.0,6622.0,110.0
4,2011-01-01 04:00:00,1,1.0,0.0,605.0,10.0


In [22]:
df_trip_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 6 columns):
date               8760 non-null datetime64[ns]
count              8760 non-null int64
cnt_reg            8760 non-null float64
cnt_cas            8760 non-null float64
duration_in_sec    8760 non-null float64
duration_in_min    8760 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 410.7 KB


__ 2. Processing the condition dataset __

In [23]:
df_cond = pd.read_csv('data/hour.csv',parse_dates=[1])

In [24]:
df_cond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
instant       17379 non-null int64
dteday        17379 non-null datetime64[ns]
season        17379 non-null int64
yr            17379 non-null int64
mnth          17379 non-null int64
hr            17379 non-null int64
holiday       17379 non-null int64
weekday       17379 non-null int64
workingday    17379 non-null int64
weathersit    17379 non-null int64
temp          17379 non-null float64
atemp         17379 non-null float64
hum           17379 non-null float64
windspeed     17379 non-null float64
casual        17379 non-null int64
registered    17379 non-null int64
cnt           17379 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(12)
memory usage: 2.3 MB


In [25]:
df_cond.rename(columns={'dteday':'date'},inplace=True)

In [26]:
df_cond.index = df_cond.date

In [27]:
df_cond.drop(['instant','date'],axis=1,inplace=True)

In [28]:
df_cond.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
date,,,,,,,,,,,,,,,
2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [29]:
# Get the data of 2011
df_cond = df_cond.loc['2011-01-01':'2011-12-31']

In [30]:
df_cond.reset_index(inplace=True)

In [31]:
df_cond.head()

,date,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [32]:
# Combined to 'date' and 'hr' column to datetime.
df_cond['date'] = df_cond.date + df_cond.hr.astype('timedelta64[h]') 

In [33]:
df_cond.head()

,date,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [34]:
df_cond_final = df_cond.drop(['season','yr','mnth','hr','weekday','workingday','casual','registered','cnt'],axis=1)

In [35]:
df_cond_final.head()

,date,holiday,weathersit,temp,atemp,hum,windspeed
0,2011-01-01 00:00:00,0,1,0.24,0.2879,0.81,0.0
1,2011-01-01 01:00:00,0,1,0.22,0.2727,0.80,0.0
2,2011-01-01 02:00:00,0,1,0.22,0.2727,0.80,0.0
3,2011-01-01 03:00:00,0,1,0.24,0.2879,0.75,0.0
4,2011-01-01 04:00:00,0,1,0.24,0.2879,0.75,0.0


In [36]:
df_cond_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8645 entries, 0 to 8644
Data columns (total 7 columns):
date          8645 non-null datetime64[ns]
holiday       8645 non-null int64
weathersit    8645 non-null int64
temp          8645 non-null float64
atemp         8645 non-null float64
hum           8645 non-null float64
windspeed     8645 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 472.9 KB


__ 3. Merge the two DataFrame: df_trip_agg and df_cond_final __

In [37]:
df_combined = pd.merge(df_trip_agg, df_cond_final, on = 'date', how= 'outer')

In [38]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8760 entries, 0 to 8759
Data columns (total 12 columns):
date               8760 non-null datetime64[ns]
count              8760 non-null int64
cnt_reg            8760 non-null float64
cnt_cas            8760 non-null float64
duration_in_sec    8760 non-null float64
duration_in_min    8760 non-null float64
holiday            8645 non-null float64
weathersit         8645 non-null float64
temp               8645 non-null float64
atemp              8645 non-null float64
hum                8645 non-null float64
windspeed          8645 non-null float64
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 889.7 KB


In [39]:
'''the null rows in df_cond_final all matches well to the null rows in df_trip_agg'''

'the null rows in df_cond_final all matches well to the null rows in df_trip_agg'

In [40]:
df_combined_final = df_combined.dropna()

In [41]:
df_combined_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8645 entries, 0 to 8759
Data columns (total 12 columns):
date               8645 non-null datetime64[ns]
count              8645 non-null int64
cnt_reg            8645 non-null float64
cnt_cas            8645 non-null float64
duration_in_sec    8645 non-null float64
duration_in_min    8645 non-null float64
holiday            8645 non-null float64
weathersit         8645 non-null float64
temp               8645 non-null float64
atemp              8645 non-null float64
hum                8645 non-null float64
windspeed          8645 non-null float64
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 878.0 KB


In [42]:
df_combined_final.to_csv('data/2011-conditions.csv', index = False)

In [43]:
df_combined_final.head(10)

,date,count,cnt_reg,cnt_cas,duration_in_sec,duration_in_min,holiday,weathersit,temp,atemp,hum,windspeed
0,2011-01-01 00:00:00,15,12.0,3.0,7745.0,129.0,0.0,1.0,0.24,0.2879,0.81,0.0000
1,2011-01-01 01:00:00,38,31.0,7.0,21907.0,365.0,0.0,1.0,0.22,0.2727,0.80,0.0000
2,2011-01-01 02:00:00,31,26.0,5.0,14126.0,235.0,0.0,1.0,0.22,0.2727,0.80,0.0000
3,2011-01-01 03:00:00,12,9.0,3.0,6622.0,110.0,0.0,1.0,0.24,0.2879,0.75,0.0000
4,2011-01-01 04:00:00,1,1.0,0.0,605.0,10.0,0.0,1.0,0.24,0.2879,0.75,0.0000
5,2011-01-01 05:00:00,1,1.0,0.0,954.0,16.0,0.0,2.0,0.24,0.2576,0.75,0.0896
6,2011-01-01 06:00:00,2,0.0,2.0,1326.0,22.0,0.0,1.0,0.22,0.2727,0.80,0.0000
7,2011-01-01 07:00:00,3,2.0,1.0,2303.0,38.0,0.0,1.0,0.20,0.2576,0.86,0.0000
8,2011-01-01 08:00:00,8,7.0,1.0,4774.0,80.0,0.0,1.0,0.24,0.2879,0.75,0.0000
9,2011-01-01 09:00:00,9,6.0,3.0,7824.0,130.0,0.0,1.0,0.32,0.3485,0.76,0.0000


In [44]:
df_combined_final.index = df_combined_final.date

In [45]:
df_combined_final.loc['2011-01-01':'2011-01-31'].cnt_reg.sum()

34340.0